# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [127]:
import numpy as np
import pandas as pd
import xlsxwriter
import scipy as stats
import requests
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [128]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [129]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
print(api_url)
data = requests.get(api_url).json()
# data

https://sandbox.iexapis.com/stable/stock/AAPL/quote?token=Tpk_059b97af715d417d9f49f50b51b1c448


## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [130]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

41.81

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [131]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [132]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
#     print(data.status_code)
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                'N/A'
            ],
            index = my_columns
        ),
            ignore_index = True
        )
#        print(symbol)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,127.20,55.48,N/A
1,AAL,15.45,-1.11,N/A
2,AAP,174.74,24.82,N/A
3,AAPL,136.41,41.57,N/A
4,ABBV,109.87,23.78,N/A
...,...,...,...,...
500,YUM,108.73,33.64,N/A
501,ZBH,164.03,-217.94,N/A
502,ZBRA,410.12,46.29,N/A
503,ZION,50.16,21.41,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [133]:
# final_dataframe.sort_values('Price-to-Earnings Ratio', ascending = False, inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis = 1, inplace = True)
# final_dataframe

## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [134]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [135]:
portfolio_input()

Enter the value of your portfolio:10000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [136]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,127.20,55.48,1
1,AAP,174.74,24.82,1
2,AAPL,136.41,41.57,1
3,ABBV,109.87,23.78,1
4,ABC,111.04,15.19,1
5,ABMD,326.68,71.38,0
6,ABT,116.48,59.09,1
7,ACN,275.57,35.05,0
8,ADBE,501.20,61.69,0
9,ADI,156.79,50.12,1


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [137]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
# print(data['AAPL']['advanced-stats']['priceToBook'])

# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
# pb_ratio = data[symbol]['quote']['peRatio']
pb_ratio = data['AAPL']['advanced-stats']['priceToBook']

# Price-to-sales ratio
# print(data[symbol]['quote']['peRatio'])
ps_ratio = data['AAPL']['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
# print(data[symbol]['quote']['peRatio'])
enterprise_value = data['AAPL']['advanced-stats']['enterpriseValue']
ebitda = data['AAPL']['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# Enterprise Value divided by Gross Profit (EV/GP)
# print(data[symbol]['quote']['peRatio'])
gross_profit = data['AAPL']['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit
print(ev_to_gross_profit)


21.41099790530459


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [138]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]
rv_dataframe = pd.DataFrame(columns = rv_columns)

In [139]:
for symbol_string in symbol_strings:
#     print(symbol_string)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']

        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
            
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
        
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,132.18,N/A,55.96,N/A,8,N/A,7.5,N/A,34.826860,N/A,14.278692,N/A,N/A
1,AAL,15.56,N/A,-1.1,N/A,-1.69,N/A,0.3888,N/A,-5.896875,N/A,1.375982,N/A,N/A
2,AAP,172.08,N/A,25.9,N/A,3.19,N/A,1.23,N/A,11.813352,N/A,2.649556,N/A,N/A
3,AAPL,138.03,N/A,41.11,N/A,36,N/A,8.14,N/A,29.057154,N/A,21.384756,N/A,N/A
4,ABBV,111.22,N/A,23.65,N/A,12.7,N/A,4.83,N/A,16.196244,N/A,9.699423,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,112.38,N/A,33.24,N/A,-4.09,N/A,5.88,N/A,23.796013,N/A,10.216994,N/A,N/A
501,ZBH,163.07,N/A,-218.96,N/A,2.9,N/A,4.82,N/A,27.562828,N/A,8.087001,N/A,N/A
502,ZBRA,414.34,N/A,48.4,N/A,11.5,N/A,5.22,N/A,31.443489,N/A,11.958464,N/A,N/A
503,ZION,49.70,N/A,21.1,N/A,1.16,N/A,2.74,N/A,11.969271,N/A,3.085493,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [140]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,207.620,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,238.520,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,9.94,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,30.180,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,50.420,N/A,14.46,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,53.000,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,29.640,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,165.190,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,1811.450,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
326,MYL,16.297,N/A,31.01,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [141]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio',
       'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe.fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [142]:
rv_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'Price-to-Earnings Ratio',
       'PE Percentile', 'Price-to-Book Ratio', 'PB Percentile',
       'Price-to-Sales Ratio', 'PS Percentile', 'EV/EBITDA',
       'EV/EBITDA Percentile', 'EV/GP', 'EV/GP Percentile', 'RV Score'],
      dtype='object')

## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [143]:
from scipy.stats import percentileofscore as score
metrics = {
    'Price-to-Earnings Ratio': 'PE Percentile',
    'Price-to-Book Ratio': 'PB Percentile',
    'Price-to-Sales Ratio': 'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile'
}
for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])
        
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,132.18,N/A,55.96,86.1386,8.00,73.2673,7.5000,78.2178,34.826860,84.9505,14.278692,78.8119,N/A
1,AAL,15.56,N/A,-1.10,14.4554,-1.69,4.55446,0.3888,2.37624,-5.896875,2.77228,1.375982,5.34653,N/A
2,AAP,172.08,N/A,25.90,48.3168,3.19,42.2772,1.2300,18.5149,11.813352,29.703,2.649556,12.4752,N/A
3,AAPL,138.03,N/A,41.11,74.0594,36.00,93.2673,8.1400,80.5941,29.057154,80.5941,21.384756,90.6931,N/A
4,ABBV,111.22,N/A,23.65,44.9505,12.70,81.5842,4.8300,64.3564,16.196244,47.7228,9.699423,61.1881,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,112.38,N/A,33.24,63.5644,-4.09,4.15842,5.8800,70.8911,23.796013,71.8812,10.216994,63.5644,N/A
501,ZBH,163.07,N/A,-218.96,1.9802,2.90,39.2079,4.8200,64.1584,27.562828,79.0099,8.087001,51.4851,N/A
502,ZBRA,414.34,N/A,48.40,80.5941,11.50,80.198,5.2200,67.3267,31.443489,82.9703,11.958464,70.495,N/A
503,ZION,49.70,N/A,21.10,40.5941,1.16,11.0891,2.7400,44.1584,11.969271,31.0891,3.085493,15.6436,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [144]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)

rv_dataframe
    


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,132.18,N/A,55.96,86.1386,8.00,73.2673,7.5000,78.2178,34.826860,84.9505,14.278692,78.8119,80.2772
1,AAL,15.56,N/A,-1.10,14.4554,-1.69,4.55446,0.3888,2.37624,-5.896875,2.77228,1.375982,5.34653,5.90099
2,AAP,172.08,N/A,25.90,48.3168,3.19,42.2772,1.2300,18.5149,11.813352,29.703,2.649556,12.4752,30.2574
3,AAPL,138.03,N/A,41.11,74.0594,36.00,93.2673,8.1400,80.5941,29.057154,80.5941,21.384756,90.6931,83.8416
4,ABBV,111.22,N/A,23.65,44.9505,12.70,81.5842,4.8300,64.3564,16.196244,47.7228,9.699423,61.1881,59.9604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,112.38,N/A,33.24,63.5644,-4.09,4.15842,5.8800,70.8911,23.796013,71.8812,10.216994,63.5644,54.8119
501,ZBH,163.07,N/A,-218.96,1.9802,2.90,39.2079,4.8200,64.1584,27.562828,79.0099,8.087001,51.4851,47.1683
502,ZBRA,414.34,N/A,48.40,80.5941,11.50,80.198,5.2200,67.3267,31.443489,82.9703,11.958464,70.495,76.3168
503,ZION,49.70,N/A,21.10,40.5941,1.16,11.0891,2.7400,44.1584,11.969271,31.0891,3.085493,15.6436,28.5149


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [145]:
rv_dataframe.sort_values('RV Score', ascending = True, inplace = True)
rv_dataframe = rv_dataframe[:10]
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,23.79,N/A,5.00,15.8416,0.4568,4.9505,0.4035,2.9703,2.662699,3.36634,0.403491,0.39604,5.50495
1,AIG,40.99,N/A,-7.51,10.6931,0.5700,5.74257,0.7730,7.12871,4.573982,4.15842,0.764174,1.58416,5.86139
2,AAL,15.56,N/A,-1.10,14.4554,-1.6900,4.55446,0.3888,2.37624,-5.896875,2.77228,1.375982,5.34653,5.90099
3,HFC,26.65,N/A,-27.10,6.73267,0.7697,6.53465,0.3004,1.38614,6.321519,5.94059,2.178052,10.495,6.21782
4,F,9.00,N/A,-17.17,8.31683,1.1700,11.5842,0.2729,0.990099,4.589494,4.35644,2.009566,9.30693,6.91089
5,MET,51.33,N/A,8.40,17.0297,0.6073,5.94059,0.6955,5.94059,4.601172,4.55446,0.690547,1.18812,6.93069
6,BA,212.50,N/A,-27.80,6.53465,-10.3900,3.16832,2.0100,33.1683,-29.841151,1.38614,-203.085129,0.19802,8.89109
7,ALL,111.10,N/A,8.00,16.4356,1.3700,17.1287,0.7501,6.53465,2.452673,3.16832,0.726125,1.38614,8.93069
8,HIG,50.86,N/A,10.95,19.901,1.0600,9.50495,0.9007,9.50495,4.445055,3.9604,0.887785,1.9802,8.9703
9,FTI,12.00,N/A,-0.92,14.6535,1.2800,14.0594,0.4131,3.16832,3.614395,3.56436,2.389374,11.8812,9.46535


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [146]:
portfolio_input()

Enter the value of your portfolio:10000


In [147]:
position_size = float(portfolio_size)/len(rv_dataframe.index)
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[row, 'Price'])
    
rv_dataframe

/Users/kevin/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,23.79,42,5.00,15.8416,0.4568,4.9505,0.4035,2.9703,2.662699,3.36634,0.403491,0.39604,5.50495
1,AIG,40.99,24,-7.51,10.6931,0.5700,5.74257,0.7730,7.12871,4.573982,4.15842,0.764174,1.58416,5.86139
2,AAL,15.56,64,-1.10,14.4554,-1.6900,4.55446,0.3888,2.37624,-5.896875,2.77228,1.375982,5.34653,5.90099
3,HFC,26.65,37,-27.10,6.73267,0.7697,6.53465,0.3004,1.38614,6.321519,5.94059,2.178052,10.495,6.21782
4,F,9.00,111,-17.17,8.31683,1.1700,11.5842,0.2729,0.990099,4.589494,4.35644,2.009566,9.30693,6.91089
5,MET,51.33,19,8.40,17.0297,0.6073,5.94059,0.6955,5.94059,4.601172,4.55446,0.690547,1.18812,6.93069
6,BA,212.50,4,-27.80,6.53465,-10.3900,3.16832,2.0100,33.1683,-29.841151,1.38614,-203.085129,0.19802,8.89109
7,ALL,111.10,9,8.00,16.4356,1.3700,17.1287,0.7501,6.53465,2.452673,3.16832,0.726125,1.38614,8.93069
8,HIG,50.86,19,10.95,19.901,1.0600,9.50495,0.9007,9.50495,4.445055,3.9604,0.887785,1.9802,8.9703
9,FTI,12.00,83,-0.92,14.6535,1.2800,14.0594,0.4131,3.16832,3.614395,3.56436,2.389374,11.8812,9.46535


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [148]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine = 'xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name = 'Value Strategy', index = False)


In [149]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [150]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
    
writer.save()

## Saving Our Excel Output
As before, saving our Excel output is very easy: